In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "llama2"

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

model = Ollama(model = MODEL)
embeddings = OllamaEmbeddings()
chat = model | parser


In [15]:
chat.invoke("tell me something funny")

'\nWhy did the scarecrow win an award? Because he was outstanding in his field! (get it?)'

In [17]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("BoardingPass-25065587.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'Qt 4.8.7', 'creator': 'wkhtmltopdf 0.12.6', 'creationdate': '2024-08-29T10:05:38+00:00', 'title': 'TUI fly Boardingpass', 'source': 'BoardingPass-25065587.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content="Please show this boarding pass at baggage check-in, at the boarding terminal and to airplane personnel.\nYOU'VE CHECKED-IN - WHAT NEXT?\n \n \n \n \nYOUR BOARDING PASS\nTRAVEL DATE\n31/08/2024\nBOARDING GATE CLOSES\n05:35\nFLIGHT DEPARTS\n06:20\n \nFLIGHT NUMBER\nTOM 2668\n \nSEAT NUMBER\n37A \nPASSENGER\nKatya Andreina \nRivas Mesa\nBOOKING NUMBER:\n25065587\nCABIN:\nEconomy\nBAGGAGE:\n25 kgs\nboarding sequence number 207\nFROM\nManchester\n \n \nTO\nMajorca\nHold Baggage\nSimply head to a bag drop desk at the \nairport – check the airport screens to find out\nwhere they \nare. If your luggage allowance is more than 25kg, remember \nto split the\nweight between several bags. Reason being, \nthe maximum weight for one piece of\ncheck

In [18]:
from langchain.prompts import PromptTemplate

template = """
Answear the question based on the context below, if you can't answear the question
reply with 'I don't know'

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(question = "there is some question", context = "there is some context")

"\nAnswear the question based on the context below, if you can't answear the question\nreply with 'I don't know'\n\nContext: there is some context\n\nQuestion: there is some question\n\n"

In [20]:
chat = prompt | model | parser
chat.invoke({"context": "I am on England and weather is very bad", "question":"what country you currently at and how is the weather?"})

' Sure! Based on the context you provided, I would answer the question as follows:\n\n"I am currently in England, and unfortunately, the weather is very bad here. It\'s raining heavily and the wind is quite strong, so I can\'t enjoy the beautiful scenery around me. I hope the weather improves soon!"'

In [22]:
chat.input_schema.schema()

/var/folders/7h/nv4vxbws0jj3pggmn4f820s00000gn/T/ipykernel_14901/3594136560.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  chat.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embedding)